In [1]:
import pandas as pd
import plotly.express as px
import aqi as aqi

In [2]:
# read in data
data_file = "waqi-covid19-airqualitydata-2020.csv"
df_2020 = pd.read_csv(data_file)
df_2020['Date'] = df_2020['Date'].astype('datetime64[ns]')

df_2019Q4 = pd.read_csv("waqi-covid19-airqualitydata-2019Q4.csv")

In [3]:
# merge data 
df = df_2020.append(df_2019Q4)
# drop non-US cities
df.drop(df[df['Country']!='US'].index, inplace = True)

In [4]:
# drop irrelevant Species
df['Specie'].unique()
bad_species = ['wind gust', 'temperature', 'wind speed', 'wind-gust', 'pressure', 'dew', 'wind-speed',
       'humidity', 'precipitation', 'wd']
        
df = df[~df.Specie.isin(bad_species)]

In [5]:
# for purpose of using aqi package
d = {}
d['pm25'] = aqi.POLLUTANT_PM25
d['pm10'] = aqi.POLLUTANT_PM10
d['no2'] = aqi.POLLUTANT_NO2_1H
d['o3'] = aqi.POLLUTANT_O3_8H
d['so2'] = aqi.POLLUTANT_SO2_1H
d['co'] = aqi.POLLUTANT_CO_8H
df = df.drop(df[(df['Specie'] == 'co') & (df['median'] == 58.0)].index)

In [6]:
def compute_iaqi(row):
    if(row['Specie'] == 'o3'):
        row['median']/= 1000
    return aqi.to_iaqi(d[row['Specie']], row['median'], algo=aqi.ALGO_EPA)
df['iaqi'] = df.apply(compute_iaqi,axis=1)

In [7]:
# take max to get final AQI 
df.sort_values(by=['City','Date'])

df_new = df.drop(['Country','Specie','count','min','max','median','variance'], axis = 1)
df_new = df_new.groupby(['Date','City']).max()

print(type(df_new.axes[1]))
def AQI(row):
    #print(df_new.loc[(row['Date'],row['City'])]["iaqi"])
    return df_new.loc[(row['Date'],row['City'])]["iaqi"]

df['AQI'] = df.apply(AQI,axis=1)

<class 'pandas.core.indexes.base.Index'>


In [8]:
df = df.sort_values(by=['City','Date'])
print(df['City'].unique())

['Albuquerque' 'Atlanta' 'Austin' 'Baltimore' 'Boise' 'Boston' 'Brooklyn'
 'Charlotte' 'Chicago' 'Columbia' 'Columbus' 'Dallas' 'Denver' 'Detroit'
 'El Paso' 'Fort Worth' 'Fresno' 'Hartford' 'Honolulu' 'Houston'
 'Indianapolis' 'Jackson' 'Jacksonville' 'Las Vegas' 'Little Rock'
 'Los Angeles' 'Madison' 'Manhattan' 'Memphis' 'Miami' 'Milwaukee'
 'Nashville' 'Oakland' 'Omaha' 'Philadelphia' 'Phoenix' 'Portland'
 'Providence' 'Queens' 'Raleigh' 'Richmond' 'Sacramento' 'Saint Paul'
 'Salem' 'Salt Lake City' 'San Antonio' 'San Diego' 'San Francisco'
 'San Jose' 'Seattle' 'Springfield' 'Staten Island' 'Tallahassee'
 'The Bronx' 'Tucson' 'Washington D.C.']


In [9]:
### City -> County 

d_tocounty = {}
d_tocounty['Albuquerque'] = 'Bernalillo'
d_tocounty['Atlanta'] = 'Fulton'
d_tocounty['Austin'] = 'Travis'
d_tocounty['Baltimore'] = 'Baltimore'
d_tocounty['Boise'] = 'Ada'
d_tocounty['Boston'] = 'Suffolk'
d_tocounty['Brooklyn'] = 'Kings'
d_tocounty['Charlotte'] = 'Mecklenburg'
d_tocounty['Chicago'] = 'Cook'
d_tocounty['Columbia'] = 'Richland'
d_tocounty['Columbus'] = 'Franklin'
d_tocounty['Dallas'] = 'Dallas'
d_tocounty['Denver'] = 'Denver'
d_tocounty['Detroit'] = 'Wayne'
d_tocounty['El Paso'] = 'El Paso'
d_tocounty['Fort Worth'] = 'Tarrant'
d_tocounty['Fresno'] = 'Fresno'
d_tocounty['Hartford'] = 'Hartford'
d_tocounty['Honolulu'] = 'Honolulu'
d_tocounty['Houston'] = 'Harris'
d_tocounty['Indianapolis'] = 'Marion'
d_tocounty['Jackson'] = 'Hinds'
d_tocounty['Jacksonville'] = 'Duval'
d_tocounty['Las Vegas'] = 'Clark'
d_tocounty['Little Rock'] = 'Pulaski'
d_tocounty['Los Angeles'] = 'Los Angeles'
d_tocounty['Madison'] = 'Dane'
d_tocounty['Manhattan'] = 'New York'
d_tocounty['Memphis'] = 'Shelby'
d_tocounty['Miami'] = 'Miami-Dade'
d_tocounty['Milwaukee'] = 'Milwaukee'
d_tocounty['Nashville'] = 'Davidson'
d_tocounty['Oakland'] = 'Alameda'
d_tocounty['Omaha'] = 'Douglas'
d_tocounty['Philadelphia'] = 'Philadelphia'
d_tocounty['Phoenix'] = 'Maricopa'
d_tocounty['Portland'] = 'Multnomah'
d_tocounty['Providence'] = 'Providence'
d_tocounty['Queens'] = 'Queens'
d_tocounty['Raleigh'] = 'Wake'
d_tocounty['Richmond'] = 'Richmond City'
d_tocounty['Sacramento'] = 'Sacramento'
d_tocounty['Saint Paul'] = 'Ramsey'
d_tocounty['Salem'] = 'Marion'
d_tocounty['Salt Lake City'] = 'Salt Lake'
d_tocounty['San Antonio'] = 'Bexar'
d_tocounty['San Diego'] = 'San Diego'
d_tocounty['San Francisco'] = 'San Francisco'
d_tocounty['San Jose'] = 'Santa Clara'
d_tocounty['Seattle'] = 'King'
d_tocounty['Springfield'] = 'Greene'
d_tocounty['Staten Island'] = 'Richmond'
d_tocounty['Tallahassee'] = 'Leon'
d_tocounty['The Bronx'] = 'Bronx'
d_tocounty['Tucson'] = 'Pima'
d_tocounty['Washington D.C.'] = 'District of Columbia'


In [10]:
# COVID data: confirmed 
df_confirmed = pd.read_csv("time_series_covid19_confirmed_US.csv")
df_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,133,133,133,133,135,135,136,136,136,136
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,11,11,11,13,13,13,14,14,14
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,788,897,903,923,974,1043,1068,1118,1213,1252
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,51,51,51,51,51,51,51,53,53,53


In [13]:
df_confirmed = df_confirmed.drop(['UID','iso2','iso3','code3','FIPS'],axis=1)
df_confirmed = df_confirmed.dropna()

In [14]:
df_confirmed.transpose()

,5,6,7,8,9,10,11,12,13,14,...,3250,3251,3253,3254,3255,3256,3257,3258,3259,3260
Admin2,Autauga,Baldwin,Barbour,Bibb,Blount,Bullock,Butler,Calhoun,Chambers,Cherokee,...,Unassigned,Unassigned,Michigan Department of Corrections (MDOC),Federal Correctional Institution (FCI),Bear River,Central Utah,Southeast Utah,Southwest Utah,TriCounty,Weber-Morgan
Province_State,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,...,Wisconsin,Wyoming,Michigan,Michigan,Utah,Utah,Utah,Utah,Utah,Utah
Country_Region,US,US,US,US,US,US,US,US,US,US,...,US,US,US,US,US,US,US,US,US,US
Lat,32.5395,30.7277,31.8683,32.9964,33.9821,32.1003,31.753,33.7748,32.9136,34.1781,...,0,0,0,0,41.5211,39.3723,38.9962,37.8545,40.1249,41.2712
Long_,-86.6441,-87.7221,-85.3871,-87.1251,-86.5679,-85.7127,-86.6806,-85.8263,-85.3907,-85.6064,...,0,0,0,0,-113.083,-111.576,-110.701,-111.442,-109.517,-111.915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/16/20,26,101,14,22,18,8,11,62,231,11,...,0,0,472,36,0,0,0,0,0,0
4/17/20,26,103,15,24,20,8,16,63,236,12,...,0,0,514,44,0,0,0,0,0,0
4/18/20,25,109,18,26,20,9,13,66,240,12,...,0,0,550,45,0,0,0,0,0,0
4/19/20,26,112,20,28,21,9,14,71,245,12,...,0,0,556,51,54,15,6,66,10,119
